In [1]:
from __future__ import print_function

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]='0,1,2,3'
import sys
import argparse
import cv2

# parser = argparse.ArgumentParser()
# parser.add_argument('-mode', type=str, help='rgb or flow')
# parser.add_argument('-save_model', type=str)
# parser.add_argument('-root', type=str)

# args = parser.parse_args()
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms
import videotransforms_attention


import numpy as np

from pytorch_i3d_attention_gaze import InceptionI3d

from z_my_egtea_dataset_attention_gaze import Egtea as Dataset

from pytorch_ranger import Ranger, RangerVA

Random Seed:  922


In [2]:
n_wkers = 2
b_sz = 6
mx_steps = 64e3

# try to set random seed
manualSeed = 922
print("Random Seed: ", manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  922


In [5]:
# def test_on_val_set(i3d, g_mode = 'rgb'):
    
#     root = './dataset/EGTEA_Gaze_plus/z_frames_' + g_mode

#     cnt = 0
#     g_num_classes = 106
#     g_offset = 8
#     g_clip_length = 16

#     val_split = './dataset/EGTEA_Gaze_plus/action_annotation/val_split.txt'

#     with open(val_split, 'r') as f:
#         lines = f.readlines()
    
#     num_instances = len(lines)
#     test_transforms = transforms.Compose([videotransforms_attention.CenterCrop(224)])
    
#     class_correct = np.zeros(g_num_classes)
#     class_total = np.zeros(g_num_classes)

#     all_scores = np.zeros((num_instances, g_num_classes))
#     gggg_labels = np.zeros(num_instances)
    
#     tot_loss = 0.0
#     tot_gaze_loss = 0.0

#     for ixx, l in enumerate(lines):
#         sp = l.split(' ')

#         vid = sp[0]
#         gt_label = int(sp[1]) - 1  # 1-106 ==> 0-105
                
#         spp = vid.split('-')
#         vid_folder = '-'.join(spp[0:3])

#         vid_path = os.path.join(root, vid_folder, vid)
#         if not os.path.exists(vid_path):
#             print('no video input')
#             continue
#         num_frames = len(os.listdir(vid_path))

#         print(vid, num_frames)

#         if g_mode == 'flow':
#             num_frames = num_frames // 2

#         if num_frames < g_clip_length:
#             print('input video too short')
#             continue

#         all_frames = load_video_frames(vid_path, 1, num_frames+1)

#         score = torch.zeros(g_num_classes).cuda()
        
#         for idx in range(1, num_frames + 1, g_offset):
#             if (idx + g_clip_length) <= num_frames + 1:

#                 frames = all_frames[idx : idx+g_clip_length]

#                 inputs = np.asarray(frames, dtype=np.float32)
#                 inputs = test_transforms(inputs)
#                 inputs = torch.from_numpy(inputs.transpose([3, 0, 1, 2]))
#                 inputs = inputs.unsqueeze(0)
#                 inputs = Variable(inputs.cuda())

#                 per_frame_logits, a_map = i3d(inputs)
                
# #                 print('len per_frame_logits', len(per_frame_logits))
# #                 print('len per_frame_logits[0, :, 0]', len(per_frame_logits[0, :, 0]))

#                 score = score + per_frame_logits[0, :, 0]
                
#                 #print("per frame logits: ", per_frame_logits[0, :, 0])
                
#                 loss, gaze_loss = calculate_loss(per_frame_logits, a_map, Variable(torch.IntTensor(gt_label).cuda()))
                
#                 tot_loss += loss
#                 tot_gaze_loss += gaze_loss
                
#             else:

#                 frames = all_frames[num_frames+1-g_clip_length : num_frames+1]

#                 inputs = np.asarray(frames, dtype=np.float32)
#                 inputs = test_transforms(inputs)
#                 inputs = torch.from_numpy(inputs.transpose([3, 0, 1, 2]))
#                 inputs = inputs.unsqueeze(0)
#                 inputs = Variable(inputs.cuda())

#                 per_frame_logits, a_map = i3d(inputs)

#                 score = score + per_frame_logits[0, :, 0]
                
#                 loss, gaze_loss = calculate_loss(per_frame_logits, a_map, Variable(torch.IntTensor(gt_label).cuda()))
                
#                 tot_loss += loss
#                 tot_gaze_loss += gaze_loss


#         if torch.max(score, dim=0)[1].item() == gt_label:
#             cnt += 1
#             class_correct[gt_label] += 1

#         class_total[gt_label] += 1

#         all_scores[ixx] = score.cpu().numpy()
#         gggg_labels[ixx] = gt_label
        
        
      
    
#     accuracy = 1.0*cnt/num_instances
#     mean_class_accuracy = sum(class_correct/class_total) / g_num_classes
    
#     return accuracy, mean_class_accuracy, tot_loss, tot_gaze_loss
    
# #         print(torch.max(score, dim=0)[1].item(), gt_label)
# #         print('xxxxxxxxxxxxxxxx')

# #     print(cnt, num_instances)
# #     print(sum(class_correct), sum(class_total))

# #     print('Accuracy')
# #     print(1.0*cnt/num_instances)

# #     print('Mean Class Accuracy')
# #     print(sum(class_correct/class_total) / g_num_classes)

In [6]:
def calculate_loss(i3d, dataloader):
    tot_loss = 0.0
    tot_gaze_loss = 0.0
    
    for data in dataloader:
        inputs, labels, gt_labels, gaze_locs = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        t = inputs.size(2)
        labels = Variable(labels.cuda())

        gt_labels = Variable(gt_labels.cuda())

        per_frame_logits, a_map = i3d(inputs)

        #print inputs.shape, per_frame_logits.shape, gaze_locs.shape #(12, 3, 16, 224, 224) (12, 106, 1) (12, 16, 2)

        # upsample to input size
        per_frame_logits = F.upsample(per_frame_logits, t, mode='linear')

        loss = F.cross_entropy(torch.max(per_frame_logits, dim=2)[0], gt_labels)
        #loss /= num_steps_per_update

        tot_loss += loss.data.item()
        #loss.backward()

        a_map = a_map.squeeze(1)

        map_N, map_t, map_height, map_width = a_map.shape
        map_Data_s = torch.zeros([map_N, map_t, map_height, map_width], dtype=torch.float32)

        for n_idx in range(map_N):
            gaze_avg1 = torch.mean(gaze_locs[n_idx, 0:8, :], dim=0)
            gaze_avg2 = torch.mean(gaze_locs[n_idx, 8:16, :], dim=0)

            x, y = gaze_avg1[0] * map_width, gaze_avg1[1] * map_height
            u, v = gaze_avg2[0] * map_width, gaze_avg2[1] * map_height
            sigma = 3
            for i in range(0, map_height):
                for j in range(0, map_width):
                    map_Data_s[n_idx, 0, i, j] = torch.exp(-1.0 * ((j - x) * (j - x) + (i - y) * (i - y)) / (2 * sigma * sigma))
                    map_Data_s[n_idx, 1, i, j] = torch.exp(-1.0 * ((j - u) * (j - u) + (i - v) * (i - v)) / (2 * sigma * sigma))

        map_Data_s = map_Data_s.cuda()

        gaze_loss = F.mse_loss(map_Data_s, a_map)
        #gaze_loss /= num_steps_per_update

        tot_gaze_loss += gaze_loss.item()

        return tot_loss, tot_gaze_loss

In [15]:
def run(init_lr=0.1, max_steps=64e3, mode='rgb', batch_size=b_sz, save_model=''):

    if mode == 'flow':
        root = './dataset/EGTEA_Gaze_plus/z_frames_flow'
    else:
        root = './dataset/EGTEA_Gaze_plus/z_frames_rgb'

    gaze_root = './dataset/EGTEA_Gaze_plus/z_gaze_data'
    train_split = './dataset/EGTEA_Gaze_plus/action_annotation/train_split1.txt'
    val_split = './dataset/EGTEA_Gaze_plus/action_annotation/val_split.txt'

    # setup dataset
    train_transforms = transforms.Compose([videotransforms_attention.RandomCrop(224),
                                           videotransforms_attention.RandomHorizontalFlip(),
    ])

    dataset = Dataset(train_split, 'training', root, gaze_root, mode, train_transforms)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=n_wkers, pin_memory=True)

    dataset_val = Dataset(val_split, 'val', root, gaze_root, mode, train_transforms)
    dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=True, num_workers=n_wkers, pin_memory=True)

    dataloaders = {'train': dataloader, 'val': dataloader_val}

    
    # setup the model
    if mode == 'flow':
        i3d = InceptionI3d(106, in_channels=2)
        i3d.load_state_dict(torch.load('models/flow_a.pt'), strict=False)
    else:
        i3d = InceptionI3d(106, in_channels=3, expanded=True)
        i3d.load_state_dict(torch.load('models/rgb_a.pt'), strict=False)

    i3d.copy_attention_weights()

    # print(i3d.attention_inc.b1a.conv3d.weight)
    # print(i3d.attention_inc_2.b1a.conv3d.weight)
    # print(i3d.attention_inc_3.b1a.conv3d.weight)

    i3d.cuda()
    i3d = nn.DataParallel(i3d)

    lr = init_lr
    #optimizer = optim.SGD(i3d.parameters(), lr=lr, momentum=0.9, weight_decay=0.0000001)
    optimizer = Ranger(i3d.parameters(), lr=0.01, weight_decay=0.0000001)
    lr_sched = optim.lr_scheduler.MultiStepLR(optimizer, [300, 1000])


    num_steps_per_update = 4 # accum gradient
    steps = 0
    # train it
    while steps < max_steps:#for epoch in range(num_epochs):
        with open('output.txt', 'a') as f:
            f.write('Step {}/{}\n'.format(steps, max_steps))
            
        print('Step {}/{}'.format(steps, max_steps))
        print(optimizer.param_groups[0]['lr'])
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                i3d.train(True)
            else:
                i3d.train(False)  # Set model to evaluate mode
                
            tot_loss = 0.0
            tot_gaze_loss = 0.0

            num_iter = 0
            optimizer.zero_grad()
            
            # Iterate over data.
            for data in dataloaders[phase]:
                num_iter += 1
                # get the inputs
                inputs, labels, gt_labels, gaze_locs = data
                
                # wrap them in Variable
                inputs = Variable(inputs.cuda())
                t = inputs.size(2)
                labels = Variable(labels.cuda())

                gt_labels = Variable(gt_labels.cuda())

                per_frame_logits, a_map = i3d(inputs)

                #print inputs.shape, per_frame_logits.shape, gaze_locs.shape #(12, 3, 16, 224, 224) (12, 106, 1) (12, 16, 2)

                # upsample to input size
                per_frame_logits = F.upsample(per_frame_logits, t, mode='linear')
                                
                loss = F.cross_entropy(torch.max(per_frame_logits, dim=2)[0], gt_labels)
                loss /= num_steps_per_update

                tot_loss += loss.data.item()
                #loss.backward()

                a_map = a_map.squeeze(1)

                map_N, map_t, map_height, map_width = a_map.shape
                map_Data_s = torch.zeros([map_N, map_t, map_height, map_width], dtype=torch.float32)

                for n_idx in range(map_N):
                    gaze_avg1 = torch.mean(gaze_locs[n_idx, 0:8, :], dim=0)
                    gaze_avg2 = torch.mean(gaze_locs[n_idx, 8:16, :], dim=0)

                    x, y = gaze_avg1[0] * map_width, gaze_avg1[1] * map_height
                    u, v = gaze_avg2[0] * map_width, gaze_avg2[1] * map_height
                    sigma = 3
                    for i in range(0, map_height):
                        for j in range(0, map_width):
                            map_Data_s[n_idx, 0, i, j] = torch.exp(-1.0 * ((j - x) * (j - x) + (i - y) * (i - y)) / (2 * sigma * sigma))
                            map_Data_s[n_idx, 1, i, j] = torch.exp(-1.0 * ((j - u) * (j - u) + (i - v) * (i - v)) / (2 * sigma * sigma))

                map_Data_s = map_Data_s.cuda()

                gaze_loss = F.mse_loss(map_Data_s, a_map)
                gaze_loss /= num_steps_per_update

                tot_gaze_loss += gaze_loss.item()

                loss_two = loss + gaze_loss*0.1
                loss_two.backward()

                if num_iter == num_steps_per_update and phase == 'train':
                    steps += 1
                    num_iter = 0
                    optimizer.step()
                    optimizer.zero_grad()
                    lr_sched.step()
                    num_steps_to_eval = 2
                    
                    if steps % num_steps_to_eval == 0:
                        with open('output.txt', 'a') as f:
                            f.write('Train Tot Loss: {:.4f} Train Gaze Loss: {:.4f} - Val Tot Loss: {:.4f} Val Gaze Loss: {:.4f}'.format(tot_loss/num_steps_to_eval, tot_gaze_loss/num_steps_to_eval, loss_val, gaze_loss_val))
                        
                        loss_val, gaze_loss_val = calculate_loss(i3d, dataloader_val)
                        
                        print('Train Tot Loss: {:.4f} Train Gaze Loss: {:.4f} - Val Tot Loss: {:.4f} Val Gaze Loss: {:.4f}'.format(tot_loss/num_steps_to_eval, tot_gaze_loss/num_steps_to_eval, loss_val, gaze_loss_val))
                        
                        if steps % 100 == 0:
                            # save model
                            torch.save(i3d.module.state_dict(), save_model+str(steps).zfill(6)+'.pt')
                        tot_loss = tot_gaze_loss = 0.

    # print(i3d.module.attention_inc.b1a.conv3d.weight)
    # print(i3d.module.attention_inc_2.b1a.conv3d.weight)
    # print(i3d.module.attention_inc_3.b1a.conv3d.weight)

# if __name__ == '__main__':
#     # need to add argparse
#     run(mode=args.mode, save_model=args.save_model, max_steps=mx_steps)

In [16]:
run()

Step 0/64000.0
0.01
----------
Train Tot Loss: 1.5506 Gaze Loss: 0.1613 - Val Tot Loss: 2.6844 Val Gaze Loss: 0.1313
Train Tot Loss: 1.0187 Gaze Loss: 0.1496 - Val Tot Loss: 2.1114 Val Gaze Loss: 0.1908
Train Tot Loss: 1.1759 Gaze Loss: 0.1852 - Val Tot Loss: 1.8076 Val Gaze Loss: 0.1763
Train Tot Loss: 0.8982 Gaze Loss: 0.2212 - Val Tot Loss: 3.5242 Val Gaze Loss: 0.2766


KeyboardInterrupt: 